In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
train_df = pd.read_csv("../../input/feedback-prize-english-language-learning/train.csv")
train_df.head(2)

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5


In [3]:
model_path_list = [
#     ('../../14_Baseline4/exp/result/14_v1_01/oof_df.csv',1), # deberta-v3-base
#     ('../../14_Baseline4/exp/result/14_v1_08/oof_df.csv',1), # deberta-base
#     ('../../14_Baseline4/exp/result/14_v1_09/oof_df.csv',1), # roberta-base
#     ('../../14_Baseline4/exp/result/14_v1_10/oof_df.csv',1), # deberta-v3-large
#     ('../../14_Baseline4/exp/result/14_v1_11/oof_df.csv',1), # xlm-roberta-base
#     ('../../14_Baseline4/exp/result/14_v1_12/oof_df.csv',1), # deberta-large
#     ('../../14_Baseline4/exp/result/14_v1_13/oof_df.csv',1), # roberta-large
#     ('../../14_Baseline4/exp/result/14_v1_14/oof_df.csv',1), # xlm-roberta-large
#     ('../../14_Baseline4/exp/result/14_v1_15/oof_df.csv',1), # deberta-v2-xlarge
#     ('../../14_Baseline4/exp/result/14_v1_16/oof_df.csv',1), # deberta-xlarge
    
#     ('../../14_Baseline4/exp/result/14_v2_01/oof_df.csv',1), # deberta-v3-base, seed200
#     ('../../14_Baseline4/exp/result/14_v2_02/oof_df.csv',1), # deberta-v3-large, seed200
    
    #('../../17_Pseudo3/exp/result/17_v1_01/oof_df.csv',1), # deberta-v3-base
    ('../../17_Pseudo3/exp/result/17_v1_02/oof_df.csv',1), # deberta-v3-base
    ('../../17_Pseudo3/exp/result/17_v1_03/oof_df.csv',1), # deberta-v3-large
    ('../../17_Pseudo3/exp/result/17_v1_04/oof_df.csv',1), # deberta-v3-base, seed200
]

oof_df_list = [
    pd.read_csv(model_path) for model_path,_ in model_path_list
]
weights_list = [
    w for _,w in model_path_list
]
weights_list = [w/sum(weights_list) for w in weights_list]

for oof_df in oof_df_list:
    oof_df = train_df[['text_id']].merge(oof_df, how='left', on='text_id')
    
num_models = len(model_path_list)

TARGET_COLS = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']

preds = 0
for oof_df, w in zip(oof_df_list, weights_list):
    preds += oof_df[TARGET_COLS].values * w

oofs = []
for oof_df in oof_df_list:
    oofs.append(oof_df[TARGET_COLS].values)

In [4]:
oof_df = oof_df_list[0].copy()
for i,col in enumerate(TARGET_COLS):
    oof_df[col] = preds[:,i]

In [5]:
import numpy as np

def calc_metric(pred, gt):
    '''
    pred : (num_data, num_labels)
    gt : (num_data, num_labels)
    '''
    score = np.sqrt(np.mean((pred - gt)**2, axis=0))
    score = score.mean()
    return score

In [6]:
score = calc_metric(pred=oof_df[TARGET_COLS].values, gt=train_df[TARGET_COLS].values)
print('CV={:.4f}'.format(score))

CV=0.4451


# Optimize class-wise

In [7]:
ensemble_predictions=np.stack(oofs)
ensemble_predictions.shape

(3, 3911, 6)

In [8]:
from skopt import gp_minimize

def run_optimize(i):
    target_col = TARGET_COLS[i]
    print("*"*50)
    print(target_col)
    print("*"*50)
    def ensemble_score(weights,return_pred=False):
        weights=np.array(weights)
        weights=weights.reshape(-1,1,1)/weights.sum()
        p=weights.reshape(-1,1,1)*ensemble_predictions[:,:,i,None]
        p=p.sum(0)
        score=calc_metric(p, train_df[[target_col]])
        if return_pred:
            return score,p
        else:
            return score   
    results = gp_minimize(ensemble_score, np.array([[0.1,1] for i in range(len(ensemble_predictions))]),
                          verbose=True,random_state=2022)
    best_weights=np.array(results['x'])/sum(results['x'])
    print(best_weights)
    # smaller is better for this metric
    score, ensemble_pred=ensemble_score(best_weights,True)
    print("score={:.4f}", score)
    return score, best_weights

In [9]:
scores = {}
best_weights = {}
for i,target_col in enumerate(TARGET_COLS):
    score, weights = run_optimize(i)
    scores[target_col] = score
    best_weights[target_col] = weights

**************************************************
cohesion
**************************************************
Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0030
Function value obtained: 0.4746
Current minimum: 0.4746
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0018
Function value obtained: 0.4747
Current minimum: 0.4746
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0017
Function value obtained: 0.4742
Current minimum: 0.4742
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0024
Function value obtained: 0.4751
Current minimum: 0.4742
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.

Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.5105
Function value obtained: 0.4742
Current minimum: 0.4741
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 0.5106
Function value obtained: 0.4742
Current minimum: 0.4741
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 0.5442
Function value obtained: 0.4741
Current minimum: 0.4741
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 0.5381
Function value obtained: 0.4753
Current minimum: 0.4741
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 0.6016
Function value obtained: 0.4741
Current minimum: 0.4741
Iteration No: 46 started. Sea

Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 0.7942
Function value obtained: 0.4741
Current minimum: 0.4741
Iteration No: 82 started. Searching for the next optimal point.
Iteration No: 82 ended. Search finished for the next optimal point.
Time taken: 0.8274
Function value obtained: 0.4747
Current minimum: 0.4741
Iteration No: 83 started. Searching for the next optimal point.
Iteration No: 83 ended. Search finished for the next optimal point.
Time taken: 0.7836
Function value obtained: 0.4750
Current minimum: 0.4741
Iteration No: 84 started. Searching for the next optimal point.
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 0.8113
Function value obtained: 0.4743
Current minimum: 0.4741
Iteration No: 85 started. Searching for the next optimal point.
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 0.8601
Function value obtained: 0.4741
Current minimum: 0.4741
Iteration No: 86 started. Sea

Iteration No: 21 ended. Search finished for the next optimal point.
Time taken: 0.2657
Function value obtained: 0.4393
Current minimum: 0.4393
Iteration No: 22 started. Searching for the next optimal point.
Iteration No: 22 ended. Search finished for the next optimal point.
Time taken: 0.3145
Function value obtained: 0.4393
Current minimum: 0.4393
Iteration No: 23 started. Searching for the next optimal point.
Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 0.3023
Function value obtained: 0.4394
Current minimum: 0.4393
Iteration No: 24 started. Searching for the next optimal point.
Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 0.3117
Function value obtained: 0.4396
Current minimum: 0.4393
Iteration No: 25 started. Searching for the next optimal point.
Iteration No: 25 ended. Search finished for the next optimal point.
Time taken: 0.2685
Function value obtained: 0.4393
Current minimum: 0.4393
Iteration No: 26 started. Sea

Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 0.5350
Function value obtained: 0.4393
Current minimum: 0.4393
Iteration No: 62 started. Searching for the next optimal point.
Iteration No: 62 ended. Search finished for the next optimal point.
Time taken: 0.6399
Function value obtained: 0.4393
Current minimum: 0.4393
Iteration No: 63 started. Searching for the next optimal point.
Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 0.6822
Function value obtained: 0.4394
Current minimum: 0.4393
Iteration No: 64 started. Searching for the next optimal point.
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 0.5682
Function value obtained: 0.4393
Current minimum: 0.4393
Iteration No: 65 started. Searching for the next optimal point.
Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 0.5789
Function value obtained: 0.4401
Current minimum: 0.4393
Iteration No: 66 started. Sea

...

Iteration No: 10 ended. Evaluation done at random point.
Time taken: 0.1988
Function value obtained: 0.4634
Current minimum: 0.4633
Iteration No: 11 started. Searching for the next optimal point.
Iteration No: 11 ended. Search finished for the next optimal point.
Time taken: 0.1987
Function value obtained: 0.4632
Current minimum: 0.4632
Iteration No: 12 started. Searching for the next optimal point.
Iteration No: 12 ended. Search finished for the next optimal point.
Time taken: 0.2211
Function value obtained: 0.4636
Current minimum: 0.4632
Iteration No: 13 started. Searching for the next optimal point.
Iteration No: 13 ended. Search finished for the next optimal point.
Time taken: 0.2816
Function value obtained: 0.4655
Current minimum: 0.4632
Iteration No: 14 started. Searching for the next optimal point.
Iteration No: 14 ended. Search finished for the next optimal point.
Time taken: 0.2693
Function value obtained: 0.4636
Current minimum: 0.4632
Iteration No: 15 started. Searching for 

Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 0.5098
Function value obtained: 0.4645
Current minimum: 0.4630
Iteration No: 51 started. Searching for the next optimal point.
Iteration No: 51 ended. Search finished for the next optimal point.
Time taken: 0.5140
Function value obtained: 0.4639
Current minimum: 0.4630
Iteration No: 52 started. Searching for the next optimal point.
Iteration No: 52 ended. Search finished for the next optimal point.
Time taken: 0.5118
Function value obtained: 0.4640
Current minimum: 0.4630
Iteration No: 53 started. Searching for the next optimal point.
Iteration No: 53 ended. Search finished for the next optimal point.
Time taken: 0.4979
Function value obtained: 0.4630
Current minimum: 0.4630
Iteration No: 54 started. Searching for the next optimal point.
Iteration No: 54 ended. Search finished for the next optimal point.
Time taken: 0.5257
Function value obtained: 0.4637
Current minimum: 0.4630
Iteration No: 55 started. Sea

Iteration No: 90 ended. Search finished for the next optimal point.
Time taken: 0.8315
Function value obtained: 0.4639
Current minimum: 0.4630
Iteration No: 91 started. Searching for the next optimal point.
Iteration No: 91 ended. Search finished for the next optimal point.
Time taken: 0.9123
Function value obtained: 0.4630
Current minimum: 0.4630
Iteration No: 92 started. Searching for the next optimal point.
Iteration No: 92 ended. Search finished for the next optimal point.
Time taken: 1.0479
Function value obtained: 0.4630
Current minimum: 0.4630
Iteration No: 93 started. Searching for the next optimal point.
Iteration No: 93 ended. Search finished for the next optimal point.
Time taken: 0.8807
Function value obtained: 0.4639
Current minimum: 0.4630
Iteration No: 94 started. Searching for the next optimal point.
Iteration No: 94 ended. Search finished for the next optimal point.
Time taken: 0.9878
Function value obtained: 0.4631
Current minimum: 0.4630
Iteration No: 95 started. Sea

Iteration No: 30 ended. Search finished for the next optimal point.
Time taken: 0.3058
Function value obtained: 0.4379
Current minimum: 0.4374
Iteration No: 31 started. Searching for the next optimal point.
Iteration No: 31 ended. Search finished for the next optimal point.
Time taken: 0.3056
Function value obtained: 0.4377
Current minimum: 0.4374
Iteration No: 32 started. Searching for the next optimal point.
Iteration No: 32 ended. Search finished for the next optimal point.
Time taken: 0.4040
Function value obtained: 0.4376
Current minimum: 0.4374
Iteration No: 33 started. Searching for the next optimal point.
Iteration No: 33 ended. Search finished for the next optimal point.
Time taken: 0.3906
Function value obtained: 0.4376
Current minimum: 0.4374
Iteration No: 34 started. Searching for the next optimal point.
Iteration No: 34 ended. Search finished for the next optimal point.
Time taken: 0.4026
Function value obtained: 0.4376
Current minimum: 0.4374
Iteration No: 35 started. Sea

Iteration No: 70 ended. Search finished for the next optimal point.
Time taken: 0.6571
Function value obtained: 0.4376
Current minimum: 0.4374
Iteration No: 71 started. Searching for the next optimal point.
Iteration No: 71 ended. Search finished for the next optimal point.
Time taken: 0.7415
Function value obtained: 0.4374
Current minimum: 0.4374
Iteration No: 72 started. Searching for the next optimal point.
Iteration No: 72 ended. Search finished for the next optimal point.
Time taken: 0.6558
Function value obtained: 0.4374
Current minimum: 0.4374
Iteration No: 73 started. Searching for the next optimal point.
Iteration No: 73 ended. Search finished for the next optimal point.
Time taken: 0.6903
Function value obtained: 0.4374
Current minimum: 0.4374
Iteration No: 74 started. Searching for the next optimal point.
Iteration No: 74 ended. Search finished for the next optimal point.
Time taken: 0.7059
Function value obtained: 0.4380
Current minimum: 0.4374
Iteration No: 75 started. Sea

In [10]:
best_weights

{'cohesion': array([0.27268845, 0.58572823, 0.14158332]),
 'syntax': array([0.45133387, 0.4617067 , 0.08695944]),
 'vocabulary': array([0.18877075, 0.40618294, 0.40504632]),
 'phraseology': array([0.32266944, 0.4504347 , 0.22689585]),
 'grammar': array([0.29365359, 0.6421331 , 0.06421331]),
 'conventions': array([0.5212504 , 0.42662456, 0.05212504])}

In [11]:
scores

{'cohesion': 0.4741163687046226,
 'syntax': 0.4392791483374523,
 'vocabulary': 0.40619412695450624,
 'phraseology': 0.4485776460177308,
 'grammar': 0.46303169475199885,
 'conventions': 0.43735596155287826}

In [12]:
CV = sum(scores.values()) / len(scores)
print("CV={:.4f}".format(CV))

CV=0.4448


# Error Analysis - Check Corr

In [13]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
train_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.695459,0.666151,0.690058,0.638689,0.666151
syntax,0.695459,1.000000,0.680562,0.725467,0.709525,0.700025
vocabulary,0.666151,0.680562,1.000000,0.735261,0.654852,0.664292
phraseology,0.690058,0.725467,0.735261,1.000000,0.719746,0.666842
grammar,0.638689,0.709525,0.654852,0.719746,1.000000,0.673301
conventions,0.666151,0.700025,0.664292,0.666842,0.673301,1.000000


In [14]:
cols = ['cohesion','syntax','vocabulary','phraseology','grammar','conventions']
oof_df[cols].corr()

,cohesion,syntax,vocabulary,phraseology,grammar,conventions
cohesion,1.000000,0.946598,0.915241,0.914134,0.826278,0.889588
syntax,0.946598,1.000000,0.940922,0.965390,0.920678,0.931520
vocabulary,0.915241,0.940922,1.000000,0.964438,0.879092,0.896178
phraseology,0.914134,0.965390,0.964438,1.000000,0.944044,0.890272
grammar,0.826278,0.920678,0.879092,0.944044,1.000000,0.859493
conventions,0.889588,0.931520,0.896178,0.890272,0.859493,1.000000
